## init

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
import jlc
import copy
import scipy
import cv2
from PIL import Image
from utils import (estimate_bias, estimate_bias_coefs,norm_quantile,
                   norm_translation,process_vol,soft_threshold,
                   get_disc_2d,inspect_fiber_vol,filter_vol,generate_results,
                   generate_results_branching,FiberGraph,get_high_density_mask)
from pathlib import Path
import os

filenames = list(Path("./data/").glob("*.tif"))
filenames = sorted([str(f) for f in filenames])
print(filenames)
nice_names = [" ".join(f.split("/")[-1].split("_")[-2:]).replace(".tif","") for f in filenames]

['data/exp1_pos0.tif', 'data/exp1_pos1.tif', 'data/exp1_pos2.tif', 'data/exp1_pos3.tif', 'data/exp2_pos0.tif', 'data/exp2_pos1.tif', 'data/exp2_pos2.tif', 'data/exp2_pos3.tif']


## old

In [2]:
#sum.png
half_of_dim1 = lambda x: x[:,:,:x.shape[1]//2]
lines = []
for filename in filenames:
    line = half_of_dim1(jlc.load_tifvol(filename)).sum((1,2))
    lines.append(line)



In [3]:
plt.plot(np.array(lines[:4]).T)
plt.plot(np.array(lines[4:]).T)
plt.legend(nice_names)
plt.xlabel("Time (frames)")
plt.ylabel("Raw Sum of Intensity")

Text(0, 0.5, 'Raw Sum of Intensity')

In [4]:
#frame_100.png
sub = [[100],range(0,684),range(0,856//2)]
frames = []
for filename in filenames:
    frame = jlc.load_tifvol(filename,sub=sub)[0]
    frames.append(frame)
montage_im = jlc.montage(frames,n_col=4,imshow=False,return_im=True)
montage_im = norm_quantile(montage_im,[0.1,0.001])
montage_im = (montage_im*255).astype(np.uint8)[:,:,0]
Image.fromarray(montage_im).save("./Figures/frame_100.png")

sub = [[-20],range(0,684),range(0,856//2)]
frames = []
for filename in filenames:
    frame = jlc.load_tifvol(filename,sub=sub)[0]
    frames.append(frame)
montage_im = jlc.montage(frames,n_col=4,imshow=False,return_im=True)
montage_im = norm_quantile(montage_im,[0.01,0.005])
montage_im = (montage_im*255).astype(np.uint8)[:,:,0]
plt.imshow(montage_im)
if False:
    Image.fromarray(montage_im).save("./Figures/frame_m20.png")

## new

In [2]:
i = 4
filename = filenames[i]

savename = filename.replace("data","data_processed3")
os.makedirs(os.path.dirname(savename),exist_ok=True)
vol_vis, best_translation = process_vol(filename,smaller=False,reset_origo_on_bad_frame=False)
jlc.save_tifvol(vol_vis,savename)




done with frame 0/718
done with frame 50/718
done with frame 100/718
done with frame 150/718
done with frame 200/718
done with frame 250/718
done with frame 300/718
done with frame 350/718
done with frame 400/718
done with frame 450/718
done with frame 500/718
done with frame 550/718
done with frame 600/718
done with frame 650/718
done with frame 700/718


In [3]:
savename = filename.replace("data","data_filtered3")
os.makedirs(os.path.dirname(savename),exist_ok=True)
vol = filter_vol(vol_vis,only_median=False,alpha=0.001,thresh_small=[0.1,0.2])
jlc.save_tifvol(vol,savename)

In [1]:
jlc.inspect_vol(np.concatenate([vol,vol_vis],axis=2))

NameError: name 'jlc' is not defined

In [9]:
generate_results(i,filenames)
generate_results_branching(i,filenames)


data/exp1_pos0
found n=31 candidate paths with len>=8
found n=31 candidate paths with mean min dist>0
data/exp1_pos0


In [4]:
i = 4
vol_vis = jlc.load_tifvol(filenames[i].replace("data","data_processed3"))
vol = jlc.load_tifvol(filenames[i].replace("data","data_filtered3"))
#jlc.inspect_vol(vol)
#jlc.inspect_vol(vol_vis)
#jlc.zoom()

In [9]:
jlc.inspect_vol(vol)

In [12]:
find_conn_func = lambda xy,nodes,node: FiberGraph.default_find_conn_func(None,xy,nodes,node,threshold=10)
fiber_graph_no_crop = FiberGraph(radius=3,find_conn_func=find_conn_func,threshold=0.2)
fiber_graph_no_crop.process_vol(vol)
fiber_graph_no_crop.remove_inactive()

fiber_graph = copy.deepcopy(fiber_graph_no_crop)
fiber_graph.process_vol(vol)

fiber_graph.cc_crop()
fiber_graph.leaf_crop()

fiber_graph.high_dens_crop(get_high_density_mask(vol[-1]))
fiber_graph.remove_inactive()

In [110]:
xlims = [90,320]
ylims = [335,200]
mf = len(vol)
colors = {"points_and_lines": "C1", "deg_1": [0,1,0], "leafs": "C0", "branches": "magenta", "recent": "C5", "root": "C4"}
sizes = {"points": 6, "lines": 1.5, "deg_1": 10, "branches": 10,"recent": 5, "root": 10}
plt.figure(figsize=(8,5))
plt.imshow(vol_vis[mf-1],cmap="gray")
fiber_graph_no_crop.plot(mpl_colors=False,color=colors["leafs"],sizes=sizes)
fiber_graph.plot(mpl_colors=False,sizes=sizes, color=colors["points_and_lines"])
fiber_graph_no_crop.plot(show_deg_1=True, show_branches=True, dont_plot=True,sizes=sizes, colors=colors)

plt.xlim(xlims)
plt.ylim(ylims)
plt.legend({"Points and Lines":colors["points_and_lines"],"Branches":colors["branches"],"Degree 1":colors["deg_1"]})
plt.axis("off")
#kwargs to be used in plt.plot to achieve the nice looking legend we want
plot_kwargs = [
    {"color":colors["points_and_lines"],"label":"nodes and vertices","markersize":sizes["points"]  ,"linewidth":sizes["lines"]    ,"fmt": ".-"},
    {"color":colors["leafs"]           ,"label":"small leafs",     "markersize":sizes["points"]  ,"linewidth":sizes["lines"]    ,"fmt": ".-"},
    {"label":"branching (deg>2)","markersize":sizes["branches"],"fmt": "o","markerfacecolor": "none","color": colors["branches"]},
    {"color":colors["deg_1"]           ,"label":"growing (deg<2)",         "markersize":sizes["deg_1"]   ,"markersize":sizes["deg_1"]   ,"fmt": "o","markerfacecolor": "none"},
    ]
                
if True: # plot the text for connected components
    #add connected components
    text_pos = [240,320]
    arrowhead_pos = [[246,286],[272,297],[204,318],[205,295]]
    plt.text(text_pos[0],text_pos[1],"connected\ncomponents",fontsize=12,color="red",ha="center",va="bottom")
    for i,arrowhead in enumerate(arrowhead_pos):
        startpos = [p2*0.5+0.5*p1 for p1,p2 in  zip(arrowhead,text_pos)]
        plt.annotate("", xy=arrowhead, xytext=startpos,arrowprops=dict(arrowstyle="->",lw=1,color="red"))
    
    linehead_pos = [[220,285],[225,285],[230,285]]
    for i,arrowhead in enumerate(linehead_pos):
        startpos = [p2*0.2+0.8*p1 for p1,p2 in  zip(arrowhead,text_pos)]
        endpos = [p2*0.5+0.5*p1 for p1,p2 in  zip(arrowhead,text_pos)]
        plt.annotate("", xy=endpos, xytext=startpos,arrowprops=dict(arrowstyle="-",lw=1,color="red"))
    plt.text(220,295,"...",fontsize=20,color="red",ha="center",va="center")

if True: # plot the text for high conf fibers
    text_pos = [180,237]
    arrowhead_pos = [[153,257],[211,220]]
    plt.text(text_pos[0],text_pos[1],"high conf.\nfibers",fontsize=12,color="red",ha="center",va="center")
    for i,arrowhead in enumerate(arrowhead_pos):
        startpos = [p2*0.6+0.4*p1 for p1,p2 in  zip(arrowhead,text_pos)]
        plt.annotate("", xy=arrowhead, xytext=startpos,arrowprops=dict(arrowstyle="->",lw=1,color="red"))
                     
jlc.simulated_legend(plot_kwargs);
plt.tight_layout()
#jlc.zoom()
if True:
    plt.savefig("./Figures/fiber_graph_vis.png",dpi=200)

In [14]:
inspect_fiber_vol(vol_vis,fiber_graph=fiber_graph,vmax=0.5)
jlc.zoom()

In [10]:
plt.imshow(vol_vis,cmap="gray")
jlc.zoom()